## 1 | Imports

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## 2 | Defined functions

In [2]:
def clean_last_update(df, c):
    for i,n in enumerate(df[c]):
        try:
            df[c][i] = int(n)
        except:
            df[c][i] = int('0')

In [3]:
def checking_nulls(df):
    # This function shows which columns have null values and returns a df with only nulls
    for c in df.columns:
        null_count = df[c].isnull().sum()
        if null_count > 0:
            print ("The column ", c, " has ", null_count, " null values")
    nulls = df[df.isna().any(axis=1)]
    return nulls.head(3)

In [4]:
def convet_to_numerical(df, list = []):
    for l in list:
        df[l] = df[l].astype('int')

## 3 | Model

### Cleaning the original data set

In [5]:
rent_df = pd.read_csv('/Users/ignaciolorenzoqueralt/Documents/Ironhack/Final Project/properties/rent/2021.11.04_cataluña_837.3249999999999_100.csv', engine='python' )
rent_df = rent_df.loc[:, ~rent_df.columns.str.contains('^Unnamed')]

In [6]:
rent_df.shape

(7984, 15)

In [7]:
clean_last_update(rent_df, c = 'last_update')

In [8]:
rent_df = rent_df.dropna()

In [9]:
checking_nulls(rent_df)

,name,area,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,geo_town,province,ccaa


In [10]:
rent_df.shape

(7835, 15)

In [11]:
numericals = ['m2', 'n_rooms', 'n_bath', 'price_m2', 'price', 'price_reduction', 'last_update']
convet_to_numerical(rent_df, numericals)

In [12]:
rent_df.dtypes

name               object
area               object
m2                  int64
n_rooms             int64
n_bath              int64
price_m2            int64
price               int64
price_reduction     int64
opportunity        object
last_update         int64
description        object
url                object
geo_town           object
province           object
ccaa               object
dtype: object

In [13]:
rent_df.head(3)

,name,area,m2,n_rooms,n_bath,price_m2,price,price_reduction,opportunity,last_update,description,url,geo_town,province,ccaa
0,"Alquiler Piso en Carrer ponent, 11. Obra nueva",Abrera,45,1,1,14,630,0,yes,0,Piso en alquiler de obra nueva en el centro de...,https://www.habitaclia.com/alquiler-piso-obra_...,abrera,barcelona,cataluña
1,"Alquiler Piso en Carrer ponent, 11. Obra nueva",Abrera,55,2,1,13,750,0,no,0,Pisos en alquiler de obra nueva en el centro d...,https://www.habitaclia.com/alquiler-piso-obra_...,abrera,barcelona,cataluña
2,"Alquiler Piso en Carrer ponent, 11. Obra nueva",Abrera,78,2,1,10,820,0,no,0,Pisos en alquiler de obra nueva en el centro d...,https://www.habitaclia.com/alquiler-piso-obra_...,abrera,barcelona,cataluña


### Defining the df for the model

In [20]:
X = rent_df[['m2', 'n_rooms', 'n_bath', 'opportunity', 'last_update', 'geo_town', 'province', 'ccaa']].copy()
y = rent_df[['price']].copy()

In [16]:
X_cat = X.select_dtypes('object')
X_num = X.select_dtypes('int')